In [1]:
from SoundDS import SoundDS
from AutoEncoder import AutoEncoderto

import torch
import torch.nn as nn

OSError: dlopen(/Users/luisdias/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so, 0x0006): Symbol not found: __ZN2at8internal15invoke_parallelExxxRKNSt3__18functionIFvxxEEE
  Referenced from: <297B3EE6-D4F0-328A-9434-2AC47E280C2D> /Users/luisdias/anaconda3/envs/pytorch/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so
  Expected in:     <9DDA9B08-547F-32F9-87F1-796977059897> /Users/luisdias/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib

In [ ]:
path_do_data_dir = 'DATA/'

# Get the list of files in the 'Piano Loops' directory using os.listdir()
piano_file_list = [path_do_data_dir + 'Piano Loops/' + filename for filename in os.listdir(path_do_data_dir + 'Piano Loops')]

# Create a pandas DataFrame with the "path_to_data" and "Instrument" columns for "Piano" entries
df = pd.DataFrame({"path_to_data": piano_file_list, "Instrument": "Piano"})

# Get the list of files in the 'Guitar Loops' directory using os.listdir()
guitar_file_list = [path_do_data_dir + 'Guitar Loops/' + filename for filename in os.listdir(path_do_data_dir + 'Guitar Loops')]

# Create a DataFrame with the "path_to_data" and "Instrument" columns for "Guitar" entries
guitar_df = pd.DataFrame({"path_to_data": guitar_file_list, "Instrument": "Guitar"})

# Concatenate the "Piano" and "Guitar" DataFrames
df = pd.concat([df, guitar_df], ignore_index=True)